In [66]:
from dataclasses import dataclass
from collections import deque

@dataclass
class Node:
    name : str
    offset : list
    channels : dict
    child : list
    parent : str
    first_in : bool
    
file_path = "walk60.bvh"
bvh_data = []
with open(file_path, 'r') as f:
    for line in f:
        bvh_data.append(line.split())

num_frames = 0
frame_time = 0
frames = []
joint_List = []
info_List = []
for line in bvh_data:
    this_line = line
    if(this_line[0] == "Hierarchy"): continue
    if(this_line[0] == "MOTION"): continue
    if(this_line[0] == "Frames:"):
        num_frames = int(this_line[1])
        continue
    if(this_line[0] == "Frame"):
        frame_time = float(this_line[2])
        continue
    if(this_line[0] == "{" or
       this_line[0] == "ROOT" or
       this_line[0] == "}" or
       this_line[0] == "End" or
       this_line[0] == "JOINT" or
       this_line[0] == "OFFSET" or
       this_line[0] == "CHANNELS"):
        info_List.append(this_line)
        continue
    frames.append(this_line)
    
# # 处理栈中数据
# def Process_data(stack):
#     some_data = []
#     while(stack):
#         info_pop = stack.pop()
#         if(type(info_pop) != Node and info_pop[0] == "{"):
#             break
#         elif(type(info_pop) != Node and info_pop[0] == "}"):
#             continue
#         else:
#             some_data.append(info_pop)
#     print(some_data)
#     now_node = Node(
#         name = None,
#         offset = [],
#         channels = {},
#         child = [],
#         parent = None,
#         first_in = True
#     )
#     last_node = None
#     for i in some_data:
#         # if(type(i) != Node and i[0] == "OFFSET"):
#         #     now_node.offset = [float(i[1]), float(i[2]), float(i[3])]
#         # elif(type(i) != Node  and i[0] == "CHANNELS"):
#         #     now_node.channels = {
#         #         "num_channels" : int(i[1]),
#         #         "channels" : [i[1:]]
#         #     }
#         # elif(type(i) != Node and i[0] == "JOINT"):
#         #     now_node.name = i[1]
#         # elif(type(i) == Node):
#         #     last_node = i
#         #     i += 1
#         if(type(i) == Node):
#             last_node = i
#             continue
#         if(type(i) != Node and i[0] == "OFFSET"):
#             now_node.offset = [float(i[1]), float(i[2]), float(i[3])]
#         elif(type(i) != Node and i[0] == "CHANNELS"):
#             now_node.channels = {
#                 "num_channels" : int(i[1]),
#                 "channels" : [i[1:]]}
#         elif(type(i) != Node and i[0] == "End" or i[0] == "Joint"):
#             if(i[0] == "End") : last_node.name = i[0] + i[1]
#             if(i[0] == "Joint") : last_node.name = i[1]
#             last_node.parent = now_node.name
#             now_node.child.append(last_node)
#             last_node.first_in = False
#             joint_List.append(last_node)
            
#         print(now_node)
#         print(last_node)
#     return now_node
        
    
            
len_info = len(info_List)
node_list = []
stack = deque()
# for i in range(len_info):
#     if(info_List[i][0] == "{"):
#         stack.append(info_List[i])
#         while(info_List[i][0] != "}"):
#             i += 1
#             stack.append(info_List[i])
#     elif(info_List[i][0] == "}"):
#         stack.pop()
#         print(stack.pop())
for i in range(len_info):
    if info_List[i][0] == "ROOT":
        root = Node(
             name = info_List[i][0] + info_List[i][1],
                offset = [],
                channels = {},
                child = [],
                parent = None,
                first_in = True)
        node_list.append(root)
        joint_List.append(root)
        continue
    if info_List[i][0] == "{" or info_List[i][0] == "OFFSET" or \
        info_List[i][0] == "CHANNELS" or info_List[i][0] == "End" or \
        info_List[i][0] == "JOINT":
            stack.append(info_List[i])
            continue
    if info_List[i][0] == "}":
        stack.append(info_List[i])

 
       
    
        


In [67]:
info_List

[['ROOT', 'RootJoint'],
 ['{'],
 ['OFFSET', '0.000000', '0.000000', '0.000000'],
 ['CHANNELS',
  '6',
  'Xposition',
  'Yposition',
  'Zposition',
  'Xrotation',
  'Yrotation',
  'Zrotation'],
 ['JOINT', 'lHip'],
 ['{'],
 ['OFFSET', '0.100000', '-0.051395', '0.000000'],
 ['CHANNELS', '3', 'Xrotation', 'Yrotation', 'Zrotation'],
 ['JOINT', 'lKnee'],
 ['{'],
 ['OFFSET', '0.000000', '-0.410000', '0.000000'],
 ['CHANNELS', '3', 'Xrotation', 'Yrotation', 'Zrotation'],
 ['JOINT', 'lAnkle'],
 ['{'],
 ['OFFSET', '0.000000', '-0.390000', '0.000000'],
 ['CHANNELS', '3', 'Xrotation', 'Yrotation', 'Zrotation'],
 ['JOINT', 'lToeJoint'],
 ['{'],
 ['OFFSET', '0.000000', '-0.050000', '0.130000'],
 ['CHANNELS', '3', 'Xrotation', 'Yrotation', 'Zrotation'],
 ['End', 'Site'],
 ['{'],
 ['OFFSET', '0.010000', '0.002000', '0.060000'],
 ['}'],
 ['}'],
 ['}'],
 ['}'],
 ['}'],
 ['JOINT', 'pelvis_lowerback'],
 ['{'],
 ['OFFSET', '0.000000', '0.093605', '0.000000'],
 ['CHANNELS', '3', 'Xrotation', 'Yrotation', 'Z

In [65]:
node_list

[Node(name='ROOTHips', offset=[], channels={}, child=[], parent=None, first_in=True)]

In [60]:
class Node:
    def __init__(self, name):
        self.name = name
        self.offset = None
        self.channels = []
        self.children = []

    def __repr__(self):
        return f"Node({self.name})"


def build_skeleton(bvh_data):
    stack = []
    root = None
    current_node = None

    for line in bvh_data:
        if line[0] == 'ROOT' or line[0] == 'JOINT':
            # 创建新节点
            node = Node(line[1])
            if current_node is not None:
                # 添加为当前节点的子节点
                current_node.children.append(node)
                stack.append(current_node)
            else:
                root = node
            current_node = node
        elif line[0] == 'OFFSET':
            # 设置偏移量
            current_node.offset = tuple(map(float, line[1:]))
        elif line[0] == 'CHANNELS':
            # 设置通道信息
            current_node.channels = line[2:]  # 跳过通道数量
        elif line[0] == '}':
            # 结束当前节点的定义，返回到父节点
            if stack:
                current_node = stack.pop()
        elif line[0] == 'End' and line[1] == 'Site':
            # 处理末端节点
            end_site_node = Node('End Site')
            current_node.children.append(end_site_node)
            stack.append(current_node)
            current_node = end_site_node

    return root

# 解析示例数据
bvh_data = [['ROOT', 'Hips'], ['{'], ['OFFSET', '0.00', '0.00', '0.00'], ['CHANNELS', '6', 'Xposition', 'Yposition', 'Zposition', 'Xrotation', 'Yrotation', 'Zrotation']]
skeleton = build_skeleton(info_List)
print(skeleton.children)


[Node(LeftUpLeg), Node(RightUpLeg)]


In [10]:
from dataclasses import dataclass
from collections import deque

@dataclass
class Node:
    name : str
    offset : list
    channels : dict
    child : list
    parent : str
    ID : int
    type : str
    
file_path = "walk60.bvh"
bvh_data = []
with open(file_path, 'r') as f:
    for line in f:
        bvh_data.append(line.split())

num_frames = 0
frame_time = 0
frames = []
joint_List = []
info_List = []
for line in bvh_data:
    this_line = line
    if(this_line[0] == "Hierarchy"): continue
    if(this_line[0] == "MOTION"): continue
    if(this_line[0] == "Frames:"):
        num_frames = int(this_line[1])
        continue
    if(this_line[0] == "Frame"):
        frame_time = float(this_line[2])
        continue
    if(this_line[0] == "{" or
       this_line[0] == "ROOT" or
       this_line[0] == "}" or
       this_line[0] == "End" or
       this_line[0] == "JOINT" or
       this_line[0] == "OFFSET" or
       this_line[0] == "CHANNELS"):
        info_List.append(this_line)
        continue
    frames.append(this_line)
    
offset_stack = deque()
channel_stack = deque()
node_stack = deque()
joint_List = []
node_id = 0

for info in info_List:
    if(info[0] == "ROOT"):
        node_stack.append(Node(
            name = "RootJoint",
            offset = None,
            channels = None,
            child = [],
            parent = None,
            ID = node_id,
            type = "root"
        ))
        node_id += 1
        continue
    elif info[0] == "{": continue
    elif info[0] == "OFFSET":
        offset_stack.append(list(map(float, info[1:])))
        continue
    elif info[0] == "CHANNELS":
        channel_stack.append({
            "channel_num" : int(info[1]),
            "channels" : info[2:]
        })
    elif info[0] == "JOINT":
        node_stack.append(Node(
            name = info[1],
            offset = None,
            channels = None,
            child = [],
            parent = None,
            ID = node_id,
            type = "joint"
        ))
        node_id += 1
        continue
    elif info[0] == "End":
        node_stack.append(Node(
            name = info[0] + info[1],
            offset = None,
            channels = None,
            parent = None,
            child = None,
            type = "End",
            ID = node_id
        ))
        node_id += 1
        continue
    elif info[0] == "}":
        pop_node = node_stack.pop()
        if pop_node.type == "End":
            top_node = node_stack[-1]
            pop_node.name = top_node.name + "_end"
            pop_node.parent = top_node.name
            pop_node.child = None
            offset_top = offset_stack.pop()
            pop_node.offset = offset_top
            pop_node.channels = None
            top_node.child.append(pop_node)
            joint_List.append(pop_node)
        elif pop_node.type == "joint":
            top_node = node_stack[-1]
            pop_offset = offset_stack.pop()
            pop_channels = channel_stack.pop()
            # print(pop_channels)
            pop_node.offset = pop_offset
            pop_node.channels = pop_channels
            pop_node.parent = top_node.name
            top_node.child.append(pop_node)
            joint_List.append(pop_node)
        elif pop_node.type == "root":
            pop_offset = offset_stack.pop()
            pop_channels = channel_stack.pop()
            # print(pop_channels)
            pop_node.offset = pop_offset
            pop_node.channels = pop_channels
            pop_node.parent = None
            joint_List.append(pop_node)
        continue
               
            

In [14]:
for node in joint_List:
    if(node.name == "RootJoint"):
        print(len(node.child))

3
